In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
#import random as rn
#import re
import shutil

In [2]:
CSV_COLUMNS = ['Survived', 'Age', 'SibSp','Parch', 'C','Q', 'S','female', 'male', 'Pclass_1', 'Pclass_2', 'Pclass_3']
FEATURES = CSV_COLUMNS[1:len(CSV_COLUMNS)]     # our feature columns
LABEL = CSV_COLUMNS[0]                          #label columns
df_train = pd.read_csv('train.csv')           # training data
df_test = pd.read_csv('test.csv')             # test data
df1 = pd.read_csv('gender_submission.csv')               # now creating a evaluation data
df_eval = pd.merge(df1,df_test,on = 'PassengerId')
df_eval.head()


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,0,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,1,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,0,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,0,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,1,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [3]:
def final_features(df):
    df.drop('Name', axis=1,inplace=True)
    df.drop('Fare', axis=1, inplace=True)
    df.drop('Ticket', axis=1, inplace=True)
    df.drop('PassengerId', axis=1, inplace=True)
    df.Embarked = df['Embarked'].fillna('S')
    dummies = pd.get_dummies(df['Embarked'])
    df = pd.concat([df,dummies],axis=1) 
    dummies1 = pd.get_dummies(df['Sex'])
    df = pd.concat([df,dummies1],axis=1)
    dummies2 = pd.get_dummies(df['Pclass'], prefix= 'Pclass')
    df = pd.concat([df,dummies2],axis=1)
    df.drop('Sex', axis=1, inplace=True)
    df.drop('Pclass', axis=1, inplace=True)
    df.drop('Embarked', axis=1, inplace=True)
    df.drop('Cabin', axis=1, inplace=True)
    # cbn = [ 'C23', 'C25', 'C27']
    #df.Cabin = df['Cabin'].fillna(rn.choice(cbn))
    df.Age = df['Age'].fillna(df.Age.median())
    df.Age = (df['Age'] - df['Age'].mean()) / (df['Age'].max() - df['Age'].min())
    df = df.dropna()
    
    return df



In [4]:
# this is our train input function

def make_train_input_fn(df, num_epochs):
  return tf.estimator.inputs.pandas_input_fn(
    x = final_features(df),
    y = df[LABEL],
    #batch_size = 128,
    num_epochs = num_epochs,
    shuffle = True,
    queue_capacity = 1000
    
  )

In [5]:
def make_feature_cols():
  return [   
    
    tf.feature_column.numeric_column('Age'),
    tf.feature_column.numeric_column('SibSp'),
    tf.feature_column.numeric_column('Parch'),
    tf.feature_column.numeric_column('C'),
    tf.feature_column.numeric_column('Q'),
    tf.feature_column.numeric_column('S'),
    tf.feature_column.numeric_column('female'),
    tf.feature_column.numeric_column('male'),
    tf.feature_column.numeric_column('Pclass_1'),
    tf.feature_column.numeric_column('Pclass_2'),
    tf.feature_column.numeric_column('Pclass_3')  
  ]

In [6]:
tf.logging.set_verbosity(tf.logging.INFO)
OUTDIR = 'trained'
shutil.rmtree(OUTDIR, ignore_errors = True) # start fresh each time
model = tf.estimator.DNNClassifier(hidden_units = [64,8, 8],feature_columns = make_feature_cols(),optimizer=tf.train.ProximalAdagradOptimizer(learning_rate=0.1,l1_regularization_strength=0.001), model_dir = OUTDIR)
model.train(input_fn = make_train_input_fn(df_train, num_epochs = 2000))


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'trained', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fda3ac01d30>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
To construct input pipelines, use the `tf.data` mod

INFO:tensorflow:loss = 27.926926, step = 5501 (0.317 sec)
INFO:tensorflow:global_step/sec: 146.976
INFO:tensorflow:loss = 36.36324, step = 5601 (0.682 sec)
INFO:tensorflow:global_step/sec: 123.697
INFO:tensorflow:loss = 43.99813, step = 5701 (0.808 sec)
INFO:tensorflow:global_step/sec: 131.495
INFO:tensorflow:loss = 33.195286, step = 5801 (0.760 sec)
INFO:tensorflow:global_step/sec: 137.505
INFO:tensorflow:loss = 38.668823, step = 5901 (0.728 sec)
INFO:tensorflow:global_step/sec: 141.993
INFO:tensorflow:loss = 33.98918, step = 6001 (0.706 sec)
INFO:tensorflow:global_step/sec: 142.954
INFO:tensorflow:loss = 33.236732, step = 6101 (0.699 sec)
INFO:tensorflow:global_step/sec: 139.726
INFO:tensorflow:loss = 37.273293, step = 6201 (0.714 sec)
INFO:tensorflow:global_step/sec: 131.252
INFO:tensorflow:loss = 35.111126, step = 6301 (0.761 sec)
INFO:tensorflow:global_step/sec: 134.772
INFO:tensorflow:loss = 35.29075, step = 6401 (0.742 sec)
INFO:tensorflow:global_step/sec: 129.751
INFO:tensorflo

INFO:tensorflow:loss = 30.328281, step = 13801 (0.768 sec)
INFO:tensorflow:global_step/sec: 122.375
INFO:tensorflow:loss = 32.841988, step = 13901 (0.813 sec)
INFO:tensorflow:Saving checkpoints for 13922 into trained/model.ckpt.
INFO:tensorflow:Loss for final step: 32.054455.


In [7]:
# this is our evaluation input function


def make_eval_input_fn(df):
  return tf.estimator.inputs.pandas_input_fn(
    x = final_features(df),
    y = df[LABEL],
    batch_size = 128,
    shuffle = False,
    queue_capacity = 1000
  )

In [8]:
#def print_eval(model, df):
  
result = model.evaluate(input_fn = make_eval_input_fn(df_eval))
print("The accuracy of our model is {} % ".format(100 * result['accuracy']) )
#print_eval(model, df_eval)

INFO:tensorflow:Calling model_fn.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-06-18T08:53:42Z
INFO:tensorflow:Graph was finalized.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from trained/model.ckpt-13922
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-06-18-08:53:42
INFO:tensorflow:Saving dict for global step 13922: accuracy = 0.83253586, accuracy_baseline = 0.6363636, auc = 0.83160615, auc_precision_recall = 0.7865119, average_loss = 2.4291418, global_step = 13922, label/mean = 0.36363637, loss = 253.84532, precision = 0.79285717, prediction/mean = 0.3908645, recall = 0.7302632
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 13922: trained/model.ckpt-13922
The accuracy of our model is 83.25358629

In [9]:
# this is our prediction input function


def make_prediction_input_fn(df):
  return tf.estimator.inputs.pandas_input_fn(
    x = final_features(df),
    y = None,
   # batch_size = 128,
    shuffle = False,
    queue_capacity = 1000
    
  )


In [10]:
# these are predictions
#data = pd.DataFrame(columns=['PassengerId', 'Survived'])
#data.index = np.arange(892, len(data) + 1)
#data['PassengerId'] = data.index
predictions = model.predict(input_fn = make_prediction_input_fn(df_test))
p_id = np.empty((0))
survived = np.empty((0))
i = 892
for items in predictions:
        result = items['probabilities'].argmax(axis=0)
        survived = np.append(survived, [result], axis=0)
        p_id = np.append(p_id, [i], axis=0)
        i = i + 1
        

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from trained/model.ckpt-13922
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [11]:
data = pd.DataFrame({'PassengerId':p_id,'Survived':survived})
data['PassengerId'] = data['PassengerId'].astype('int')
data['Survived'] = data['Survived'].astype('int')
data.to_csv('submission.csv')
print(data.describe())


       PassengerId    Survived
count   418.000000  418.000000
mean   1100.500000    0.334928
std     120.810458    0.472531
min     892.000000    0.000000
25%     996.250000    0.000000
50%    1100.500000    0.000000
75%    1204.750000    1.000000
max    1309.000000    1.000000
